In [176]:
import os
import pandas as pd
from getpass import getuser
import re
import numpy as np

In [177]:

USER = getuser()
INPUT_DIR = f"C:/Users/{USER}/Documents/GitHub/tennis-homophily/data/atp/"

# Collect all ranking_doubles files
files = [
    f for f in os.listdir(INPUT_DIR)
    if f.startswith("ranking_doubles_") and f.endswith(".xlsx")
]

print(f"Found {len(files)} files.")

dfs = []
for file in sorted(files):
    path = os.path.join(INPUT_DIR, file)
    
    try:
        df = pd.read_excel(path)

        # Drop unwanted columns if they exist
        cols_to_drop = ["Unnamed: 15", "Player Profile Link"]
        df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors="ignore")
        
        # Standardize column names: lower case + replace space with -
        df.columns = df.columns.str.lower().str.replace(" ", "-", regex=False)

        dfs.append(df)
        print(f"Processed: {file}")

    except Exception as e:
        print(f"Error reading {file}: {e}")


# Count rows in each single df
rows_per_df = [len(d) for d in dfs]

# Total expected rows
expected_rows = sum(rows_per_df)

print("Rows per file:", rows_per_df)
print("Expected total rows:", expected_rows)

# Merge
merged_df = pd.concat(dfs, ignore_index=True)

actual_rows = len(merged_df)

print("Actual merged rows:", actual_rows)

if actual_rows == expected_rows:
    print("✅ Row count matches! No rows lost or duplicated during merge.")
else:
    print("❌ Mismatch! Expected:", expected_rows, "but got:", actual_rows)



Found 23 files.
Processed: ranking_doubles_2018-01-15.xlsx
Processed: ranking_doubles_2018-05-21.xlsx
Processed: ranking_doubles_2018-07-02.xlsx
Processed: ranking_doubles_2018-09-24.xlsx
Processed: ranking_doubles_2019-01-14.xlsx
Processed: ranking_doubles_2019-05-20.xlsx
Processed: ranking_doubles_2019-07-01.xlsx
Processed: ranking_doubles_2019-08-26.xlsx
Processed: ranking_doubles_2020-01-20.xlsx
Processed: ranking_doubles_2020-08-31.xlsx
Processed: ranking_doubles_2020-09-21.xlsx
Processed: ranking_doubles_2021-02-08.xlsx
Processed: ranking_doubles_2021-05-24.xlsx
Processed: ranking_doubles_2021-06-28.xlsx
Processed: ranking_doubles_2021-08-30.xlsx
Processed: ranking_doubles_2022-01-17.xlsx
Processed: ranking_doubles_2022-05-16.xlsx
Processed: ranking_doubles_2022-06-27.xlsx
Processed: ranking_doubles_2022-08-22.xlsx
Processed: ranking_doubles_2023-01-16.xlsx
Processed: ranking_doubles_2023-05-22.xlsx
Processed: ranking_doubles_2023-07-03.xlsx
Processed: ranking_doubles_2023-08-28.

# clear players'names

In [178]:
# All columns that represent players
player_cols = [col for col in merged_df.columns if col.startswith("player")]

def split_player_initial_surname(x):
    """
    Clean a raw player string and return:
    (first_initial, surname)

    Examples:
      'L. Kubot'            -> ('l', 'kubot')
      'l.kubot'             -> ('l', 'kubot')
      'Bob Bryan'           -> ('b', 'bryan')
      '1O. Marach'          -> ('m', 'marach')
      '-1R. A. Harrison'    -> ('a', 'harrison')
      'Mannarino'           -> ('m', 'mannarino')
    """
    if pd.isna(x):
        return (None, None)

    s = str(x).lower().strip()

    # remove leading junk such as "-1R.", "1O.", numbers, etc.
    s = re.sub(r"^[^a-z]+", "", s)

    # normalize dots to spaces: "l.kubot" -> "l kubot"
    s = s.replace(".", " ")

    # collapse multiple spaces
    parts = s.split()
    if len(parts) == 0:
        return (None, None)

    if len(parts) == 1:
        surname = parts[0]
        initial = surname[0]
        return (initial, surname)

    # more than one token: use first token's first letter + last token as surname
    initial = parts[0][0]
    surname = parts[-1]
    return (initial, surname)


# Apply to each player column and create two new columns
for col in player_cols:
    initials, surnames = zip(*merged_df[col].apply(split_player_initial_surname))
    merged_df[f"{col}_initial"] = initials
    merged_df[f"{col}_surname"] = surnames


In [179]:
merged_df.drop(columns=player_cols, inplace=True)


# split w-l columns in two separate variables 

In [180]:
# # Identify all W-L columns in merged_df
# wl_cols = [col for col in merged_df.columns if col.startswith("w-l-")]

# def split_wl(value):
#     """
#     Converts strings like '24 - 28' or '435-305' into (24, 28).
#     Returns (None, None) if parsing fails.
#     """
#     if pd.isna(value):
#         return (None, None)

#     value = str(value).strip()

#     # Extract two numbers
#     match = re.findall(r"\d+", value)
#     if len(match) == 2:
#         return int(match[0]), int(match[1])
#     else:
#         return (None, None)

# for col in wl_cols:
#     # Example: w-l-ytd → ytd
#     suffix = col.replace("w-l-", "")

#     # Create new columns
#     merged_df[f"w-{suffix}"], merged_df[f"l-{suffix}"] = zip(
#         *merged_df[col].apply(split_wl)
#     )

#     # Remove the original column
#     merged_df.drop(columns=col, inplace=True)

# print("W-L columns successfully split.")


In [181]:
# --- Win-ratio from separate W and L columns --- #
def wl_to_ratio_from_cols(wins, losses):
    """
    Calcola il win ratio da due colonne separate:
        wins   = numero vittorie
        losses = numero sconfitte

    Restituisce wins / (wins + losses), oppure NaN se:
        - wins o losses sono NaN/non numerici
        - wins + losses == 0
    """
    wins = pd.to_numeric(wins, errors="coerce")
    losses = pd.to_numeric(losses, errors="coerce")

    total = wins + losses
    ratio = wins / total

    ratio[(total <= 0) | total.isna()] = np.nan
    return ratio


# --- 1. Individua tutte le colonne win-loss --- #
# matches: doubles_w-l-ytd, singles_wl_career, ecc.

wl_cols = [
    col for col in merged_df.columns
    if re.search(r"(?:^|[_-])w[-_]?l(?:[_-]|$)", col.lower())
]


def split_wl(value):
    """
    Converte stringhe tipo '24-28' o '435 - 305' in (wins, losses).
    """
    if pd.isna(value):
        return (np.nan, np.nan)

    match = re.findall(r"\d+", str(value))
    if len(match) == 2:
        return int(match[0]), int(match[1])
    return (np.nan, np.nan)


# Manteniamo traccia delle coppie wins/losses create
wl_pairs = []

for col in wl_cols:
    # normalizza nome base: w-l / wl → win_loss
    base = re.sub(r"w[-_]?l", "win_loss", col, flags=re.IGNORECASE)

    wins_col = base.replace("win_loss", "wins")
    losses_col = base.replace("win_loss", "losses")

    merged_df[wins_col], merged_df[losses_col] = zip(
        *merged_df[col].apply(split_wl)
    )

    wl_pairs.append((wins_col, losses_col))

    merged_df.drop(columns=col, inplace=True)

print("Win–loss columns successfully split.")


# --- 2. Crea le colonne di win ratio --- #

for wins_col, losses_col in wl_pairs:
    ratio_col = wins_col.replace("wins", "win_ratio")

    merged_df[ratio_col] = wl_to_ratio_from_cols(
        merged_df[wins_col],
        merged_df[losses_col]
    )

print("Win-ratio columns successfully created.")


Win–loss columns successfully split.
Win-ratio columns successfully created.


# extract the missing values of dob from age 

In [182]:
def extract_dob_from_age(age_value):
    """
    Extracts the date inside parentheses in the format YYYY/MM/DD.
    Example: '42 (1983/09/23)' → '1983/09/23'
    """
    if pd.isna(age_value):
        return None

    age_str = str(age_value)

    # Look for a date inside parentheses
    match = re.search(r"\((\d{4}/\d{2}/\d{2})\)", age_str)
    if match:
        return match.group(1)
    return None

# Apply only to rows where dob is missing
missing_dob_mask = merged_df["dob"].isna()

merged_df.loc[missing_dob_mask, "dob"] = merged_df.loc[missing_dob_mask, "age"].apply(
    extract_dob_from_age
)

print("DOB values updated where possible.")


DOB values updated where possible.


In [183]:
merged_df = merged_df.drop(columns=["age"])


# weight and height split in two different measures 


In [184]:
# ---------- WEIGHT CLEANING ----------
def extract_weight_lbs(x):
    if pd.isna(x):
        return None
    match = re.search(r"(\d+)\s*lbs", str(x).lower())
    return int(match.group(1)) if match else None

def extract_weight_kg(x):
    if pd.isna(x):
        return None
    match = re.search(r"\((\d+)\s*kg\)", str(x).lower())
    return int(match.group(1)) if match else None

if "weight" in merged_df.columns:
    merged_df["weight-lbs"] = merged_df["weight"].apply(extract_weight_lbs)
    merged_df["weight-kg"]  = merged_df["weight"].apply(extract_weight_kg)
    merged_df.drop(columns=["weight"], inplace=True)


# ---------- HEIGHT CLEANING ----------
def extract_height_ft(x):
    if pd.isna(x):
        return None
    match = re.search(r"(\d+)'\s*(\d+)\"", str(x).lower())
    return int(match.group(1)) if match else None

def extract_height_in(x):
    if pd.isna(x):
        return None
    match = re.search(r"(\d+)'\s*(\d+)\"", str(x).lower())
    return int(match.group(2)) if match else None

def extract_height_cm(x):
    if pd.isna(x):
        return None
    match = re.search(r"\((\d+)\s*cm\)", str(x).lower())
    return int(match.group(1)) if match else None

if "height" in merged_df.columns:
    merged_df["height-ft"] = merged_df["height"].apply(extract_height_ft)
    merged_df["height-in"] = merged_df["height"].apply(extract_height_in)
    merged_df["height-cm"] = merged_df["height"].apply(extract_height_cm)
    merged_df.drop(columns=["height"], inplace=True)

print("Weight and height cleaned successfully.")


Weight and height cleaned successfully.


# variable experience = year - turned pro

In [185]:
merged_df.columns

Index(['rank', 'tourns', 'tournament', 'year', 'dateweek',
       'doubles_titles-ytd', 'doubles_titles-career', 'dob', 'turned-pro',
       'country', 'birthplace', 'plays', 'coach', 'singles_career_high_rank',
       'singles_date_career_high_rank', 'singles_titles_career',
       'player_initial', 'player_surname', 'doubles_wins-ytd',
       'doubles_losses-ytd', 'doubles_wins-career', 'doubles_losses-career',
       'singles_wins_career', 'singles_losses_career', 'doubles_win_ratio-ytd',
       'doubles_win_ratio-career', 'singles_win_ratio_career', 'weight-lbs',
       'weight-kg', 'height-ft', 'height-in', 'height-cm'],
      dtype='object')

In [186]:
# Create experience variable only if both columns exist
if "year" in merged_df.columns and "turned-pro" in merged_df.columns:
    merged_df["experience_double"] = merged_df["year"] - merged_df["turned-pro"]
else:
    print("Column 'year' or 'turned-pro' is missing.")


# variables single rank

In [187]:
# 1) Parse dates (coerce invalid to NaT)
merged_df["dateweek_dt"] = pd.to_datetime(merged_df["dateweek"], format="%Y-%m-%d", errors="coerce")
merged_df["singles_rank_date_dt"] = pd.to_datetime(
    merged_df["singles_date_career_high_rank"], format="%Y.%m.%d", errors="coerce"
)
merged_df["dob_dt"] = pd.to_datetime(merged_df["dob"], format="%Y/%m/%d", errors="coerce")

# 2) Difference in days: dateweek - rank date
merged_df["days_since_single_career_high_rank"] = (
    merged_df["dateweek_dt"] - merged_df["singles_rank_date_dt"]
).dt.days

# 3) Age when they reached career-high rank (in years)
merged_df["age_at_single_career_high_rank_years"] = (
    (merged_df["singles_rank_date_dt"] - merged_df["dob_dt"]).dt.days / 365.25
)

# Optional: also store as whole days (more “exact”)
merged_df["age_at_career_high_rank_days"] = (
    merged_df["singles_rank_date_dt"] - merged_df["dob_dt"]
).dt.days


In [188]:
# only meaningful if peak is in the past (>=0 days) and peak rank <= 100
is_top100_peak = merged_df["singles_career_high_rank"].le(100)
peak_in_past   = merged_df["days_since_single_career_high_rank"].ge(0)

d = merged_df["days_since_single_career_high_rank"]

merged_df["top100_within_1y"] = (
    is_top100_peak & peak_in_past & (d <= 365)
).astype(int)

merged_df["top100_within_5y"] = (
    is_top100_peak & peak_in_past & (d > 365) & (d <= 5 * 365)
).astype(int)

merged_df["top100_within_10y"] = (
    is_top100_peak & peak_in_past & (d > 5 * 365) & (d <= 10 * 365)
).astype(int)



In [189]:

# Make sure ranks are numeric
merged_df["rank_num"] = (
    merged_df["rank"]
    .astype(str)
    .str.extract(r"(\d+)", expand=False)
    .astype(float)
)

merged_df["singles_career_high_rank_num"] = pd.to_numeric(
    merged_df["singles_career_high_rank"], errors="coerce"
)

base_gap = merged_df["rank_num"] - merged_df["singles_career_high_rank_num"]

d = merged_df["days_since_single_career_high_rank"]
is_top100_peak = merged_df["singles_career_high_rank_num"].le(100)
peak_in_past   = d.ge(0)

cond_1y  = is_top100_peak & peak_in_past & (d <= 365)
cond_5y  = is_top100_peak & peak_in_past & (d > 365) & (d <= 5 * 365)
cond_10y = is_top100_peak & peak_in_past & (d > 5 * 365) & (d <= 10 * 365)

merged_df["gap_1y"]  = np.where(cond_1y,  base_gap, np.nan)
merged_df["gap_5y"]  = np.where(cond_5y,  base_gap, np.nan)
merged_df["gap_10y"] = np.where(cond_10y, base_gap, np.nan)

merged_df["abs_gap_1y"]  = merged_df["gap_1y"].abs()
merged_df["abs_gap_5y"]  = merged_df["gap_5y"].abs()
merged_df["abs_gap_10y"] = merged_df["gap_10y"].abs()

merged_df["single_specialist"] = (
    merged_df["singles_career_high_rank_num"].le(100) &
    base_gap.gt(30)
).astype(int)




# birthplace

In [190]:
def split_birthplace(x):
    if pd.isna(x):
        return (None, None)
    
    parts = str(x).split(",")
    
    if len(parts) == 2:
        city = parts[0].strip()
        country = parts[1].strip()
        return city, country
    else:
        # If the format is not city, country → return entire string as city
        return str(x).strip(), None

# Apply split
merged_df["birthplace-city"], merged_df["birthplace-country"] = zip(
    *merged_df["birthplace"].apply(split_birthplace)
)

# Optional: drop the original column
merged_df.drop(columns=["birthplace"], inplace=True)


# hand and backhand 

In [191]:
def split_plays(x):
    if pd.isna(x):
        return (None, None)
    
    parts = str(x).split(",")
    
    if len(parts) == 2:
        hand = parts[0].strip()
        backhand = parts[1].strip()
        return hand, backhand
    else:
        # If format unexpected, store everything in "hand"
        return str(x).strip(), None

merged_df["hand"], merged_df["backhand"] = zip(
    *merged_df["plays"].apply(split_plays)
)

# Optional: drop the original column
merged_df.drop(columns=["plays"], inplace=True)


# colonial legacy

In [192]:
colonial_map = {
    # Spanish legacy
    "Spain": "Spanish",
    "Argentina": "Spanish",
    "Chile": "Spanish",
    "Peru": "Spanish",
    "Mexico": "Spanish",
    "Colombia": "Spanish",
    "Uruguay": "Spanish",
    "Venezuela": "Spanish",
    "Ecuador": "Spanish",
    "Bolivia": "Spanish",
    "Paraguay": "Spanish",
    "Costa Rica": "Spanish",
    "Panama": "Spanish",
    "El Salvador": "Spanish",

    # Portuguese legacy
    "Portugal": "Portuguese",
    "Brazil": "Portuguese",

    # Dutch legacy
    "Netherlands": "Dutch",
    "Indonesia": "Dutch",

    # French legacy
    "France": "French",
    "Senegal": "French",
    "Ivory Coast": "French",
    "Côte d'Ivoire": "French",
    "Mali": "French",
    "Cameroon": "French",
    "Morocco": "French",
    "Tunisia": "French",
    "Monaco": "French",

    # British legacy
    "United Kingdom": "British",
    "Great Britain": "British",
    "UK": "British",
    "England": "British",
    "Australia": "British",
    "United States": "British",
    "USA": "British",
    "Canada": "British",
    "India": "British",
    "South Africa": "British",
    "New Zealand": "British"
}


In [193]:
merged_df["colonial_legacy"] = (
    merged_df["country"]
    .map(colonial_map)
    .fillna("None")
)


# federal legacy

In [194]:
federal_map = {
    # -------------------------
    # Yugoslavia (SFRY)
    # -------------------------
    "Croatia": "Yugoslavia",
    "Serbia": "Yugoslavia",
    "Slovenia": "Yugoslavia",
    "Bosnia and Herzegovina": "Yugoslavia",
    "Bosnia & Herzegovina": "Yugoslavia",
    "Bosnia": "Yugoslavia",
    "Montenegro": "Yugoslavia",
    "North Macedonia": "Yugoslavia",
    "Macedonia": "Yugoslavia",

    # -------------------------
    # Czechoslovakia
    # -------------------------
    "Czech Republic": "Czechoslovakia",
    "Czechia": "Czechoslovakia",
    "Slovakia": "Czechoslovakia",
    "Slovak Republic": "Czechoslovakia",

    # -------------------------
    # Soviet Union (USSR)
    # -------------------------
    "Russia": "Soviet Union",
    "Russian Federation": "Soviet Union",
    "Ukraine": "Soviet Union",
    "Belarus": "Soviet Union",
    "Moldova": "Soviet Union",

    "Estonia": "Soviet Union",
    "Latvia": "Soviet Union",
    "Lithuania": "Soviet Union",

    "Georgia": "Soviet Union",
    "Armenia": "Soviet Union",
    "Azerbaijan": "Soviet Union",

    "Kazakhstan": "Soviet Union",
    "Uzbekistan": "Soviet Union",
    "Turkmenistan": "Soviet Union",
    "Kyrgyzstan": "Soviet Union",
    "Tajikistan": "Soviet Union",

    # -------------------------
    # Nordic countries
    # -------------------------
    "Norway": "Nordic",
    "Sweden": "Nordic",
    "Denmark": "Nordic",
    "Finland": "Nordic",
    "Iceland": "Nordic",

    # -------------------------
    # Germanic countries (your definition)
    # -------------------------
    "Germany": "Germanic",
    "Austria": "Germanic",
    "Netherlands": "Germanic",
    "Belgium": "Germanic",  

    # -------------------------
    # Eastern Europe (non-federal)
    # -------------------------
    "Poland": "Eastern Europe",
    "Hungary": "Eastern Europe",
    "Romania": "Eastern Europe",
    "Bulgaria": "Eastern Europe",
    "Greece": "Eastern Europe" 
}


In [195]:
merged_df["federal_legacy"] = (
    merged_df["country"]
    .map(federal_map)
    .fillna("None")
)


# export output

In [196]:
# Save output
output_path = os.path.join(INPUT_DIR, "men_rankings.xlsx")
merged_df.to_excel(output_path, index=False)

print(f"\nMerged file saved as: {output_path}")


Merged file saved as: C:/Users/aldi/Documents/GitHub/tennis-homophily/data/atp/men_rankings.xlsx
